In [350]:
from typing import List
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

CONFIRMED_CASES_URL = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data" \
                      f"/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv "

"""
When downloading data it's better to do it in a global scope instead of a function.
This speeds up the tests significantly
"""
confirmed_cases = pd.read_csv(CONFIRMED_CASES_URL, error_bad_lines=False)

In [421]:
def poland_cases_by_date(day: int, month: int, year: int = 2020) -> int:
    date = f'{month}/{day}/{year%100}'    
    poland = confirmed_cases[lambda df: df['Country/Region']=='Poland']
    by_day = poland.loc[:,date].values[0]
    return by_day
poland_cases_by_date(7,3,2020)



5

In [420]:
def top5_countries_by_date(day: int, month: int, year: int = 2020) -> List[str]:
    cr = 'Country/Region'
    ddate = f'{month}/{day}/{year%100}'
    countries = confirmed_cases.loc[:,[ddate,cr]].groupby(cr).sum()
    countries = countries.sort_values(ddate, ascending=False).index[:5]
    return list(countries)
top5_countries_by_date(22,1,2020)

['China', 'Japan', 'Thailand', 'Korea, South', 'Taiwan*']

In [474]:
def new_cases_count(day: int, month: int, year: int = 2020) -> int:
    date = f'{month}/{day}/{year%100}'
    cr='Country/Region'
    grouped = confirmed_cases
    new_cases = grouped.loc[lambda df:df.shift(periods=1, axis=1)[date]!=df[date]]
    return new_cases
new_cases_count(10,3)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,today,yesterday,no_new_cases
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,53,59,70,75,82,114,147,50,50,False
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,581,639,639,701,773,839,825,502,461,True
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,160,178,178,200,212,226,243,150,138,True
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,129,149,149,197,238,428,566,99,93,True
6,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,55,65,65,92,112,134,171,38,28,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,"Los Angeles, CA",US,34.0522,-118.2437,0,0,0,0,1,1,...,0,0,0,0,0,0,0,14,14,False
348,"King County, WA",US,47.6062,-122.3321,1,1,1,1,1,1,...,0,0,0,0,0,0,0,83,71,True
349,"Cook County, IL",US,41.7377,-87.6976,0,0,1,1,1,1,...,0,0,0,0,0,0,0,7,6,True
404,United Kingdom,United Kingdom,55.3781,-3.4360,0,0,0,0,0,0,...,382,456,456,798,1140,1140,1543,273,206,True


In [472]:
def no_new_cases_count(day: int, month: int, year: int = 2020) -> int:
    date = f"{month}/{day}/{year % 100}"
    confirmed_cases["today"] = confirmed_cases[date].shift(periods=0, fill_value=0)
    confirmed_cases["yesterday"] = confirmed_cases.shift(periods=1, fill_value=0, axis=1)[date]
    confirmed_cases["no_new_cases"] = confirmed_cases["today"] != confirmed_cases["yesterday"]
    no_new_cases = confirmed_cases.no_new_cases.sum()
    return no_new_cases
no_new_cases_count(8,3)

116